In [64]:
import numpy as np
from numpy import cos, sin
from scipy.optimize import minimize
from easydict import EasyDict as edict

In [ ]:
CAM_HEIGHT=1.0
FX = 1.0
FY = 1.0
PX = 0
PY = 0

def wrap(rad):
    """wrap a angle between 0 to 2pi

    Args:
        rad (_type_): _description_
    """
    twopi = 2*np.pi
    while rad >= twopi:
        rad-=twopi
    
    while rad<0:
        rad+=twopi
    return rad

def forward_ekf_single_pass(x,P,u,y,R,
                            landmark_ind):
    """compute one pass of ekf update

    Args:
        x (_type_): state mean
        P: state cov matrix
        u (_type_): control input
        y : new measurement
        R : measurement noise covariance
        jacobians (_type_): dictionary coding various jacobians
        state_transition : state transition function
        observation_model : measurement function
    """

    # F: jacobian of state transition w.r.t x
    # G: jacobian of state transition w.r.t process noise
    F,G = state_transition_jacobian(x) # linearize around the previous mean

    # ======= 1. prediction step ========= #
    x_pred = state_transition(x, u, 0) # zero noise
    y_pred = observation_model(x_pred)

    # jacobian of measurement model w.r.t state
    H = observation_model_jacobian(x_pred,landmark_ind)

    P_ = np.linalg.multi_dot(F, P, F.T) + np.linalg.multi_dot(G)
    S = np.linalg.multi_dot(H, P_, H.T) + R

    # Kalman gain
    K = np.linalg.multi_dot(P_, H.T, np.linalg.inv(S))

    # ======= 2. correction step ========= #
    n = x.shape[0]
    x_next = x + K.dot(y-y_pred)
    P_next = (np.eye(n)-K.dot(H)).dot(P)

    out = edict()
    out.x = x
    out.P = P
    out.x_pred = x_pred
    out.x_next = x_next
    out.P_next = P_next
    out.u = u
    out.R = R
    out.F = F
    out.G = G
    out.H = H
    out.y_pred = y_pred
    out.P_ = P_
    out.S = S
    out.K = K

    return out

def state_transition(x, u, noise, dt=0.1):
    """state transition of a omnidirectional wheeled robot commanded by linear 
    velocity in x, y direction, and an angular velocity. In the simulation,
    it is simply implemented as two linear joints and one rotational joint

    Args:
        x (_type_): state vector. the first 3 elements correspond to robot pose
        u (_type_): control signal, vx, vy and wz
        noise (_type_): _description_
    """
    if not isinstance(noise, np.ndarray):
        noise = noise*np.ones(3)

    # unpack
    vx, vy, wz = u

    # update
    x[0] = x[0]+vx*dt+noise[0]
    x[1] = x[1]+vy*dt+noise[1]
    x[2] = wrap(x[2]+wz*dt)+noise[2]

    return x

def state_transition_jacobian(x):
    n = x.shape[0]
    jac = np.zeros((n,n))
    jac[0,0] = 1
    jac[1,1] = 1
    jac[2,2] = 1

    F = jac
    G = jac

    return F,G

def cam_pose_matrix(x, hc=CAM_HEIGHT):
    # unpack robot pose
    xr, yr, theta = x[:2]

    # robot pose matrix
    Rrobot = np.array([[cos(theta), -sin(theta),0],
                       [sin(theta), cos(theta),0],
                       [0,0,1]])
    trobot = np.array([[xr],[yr],0])

    Trobot_inv = np.concatenate([Rrobot.T,-Rrobot.T.dot(trobot)],axis=1)
    Trobot_inv = np.concatenate([Trobot_inv, np.array([[0,0,0,1]])],axis=0)

    Tbc_inv = np.array([[0,-1, 0,  0],
                        [0, 0,-1, hc],
                        [1, 0, 0,  0],
                        [0, 0, 0,  1]])
    
    Tcam_inv = np.linalg.multi_dot([Tbc_inv, Trobot_inv])
    return Tcam_inv

def get_cam_frame_coord(x, Tcam_inv, ind):
    xs,ys,zs = x[3+ind*3:3+ind*3+3]
    h_coord = np.array([xs,ys,zs,1])[:,np.newaxis]
    xs_hat, ys_hat, zs_hat, _ = Tcam_inv.dot(h_coord).flatten()
    return xs_hat, ys_hat, zs_hat

def project(xs_hat, ys_hat, zs_hat, fx=FX,fy=FY,px=PX,py=PY):
    ux = fx*xs_hat/zs_hat+px
    uy = fx*ys_hat/zs_hat+py
    return ux, uy

def observation_model(x,landmark_indices=None):
    """ get the camera projection points of the landmarks

    Args:
        x (_type_): _description_
        camera_height (_type_): _description_
        calibration_matrix (_type_): _description_
    """
    if landmark_indices is None:
        n = x.shape
        landmark_indices = [_ for _ in range((n-3)/3)]
    Tcam_inv = cam_pose_matrix(x)
    n_l = len(landmark_indices)
    out = np.zeros((n_l*2,1))
    for k,ind in enumerate(landmark_indices):
        xs_hat, ys_hat, zs_hat,_ = get_cam_frame_coord(x, Tcam_inv, ind)
        ux,uy = project(xs_hat, ys_hat, zs_hat)
        out[2*k] = ux
        out[2*k+1] = uy

    return out

def observation_model_jacobian(x,
                               landmark_ind,
                               fx=FX,fy=FY):
    Tcam_inv = cam_pose_matrix(x)
    ind = landmark_ind

    xs_hat, ys_hat, zs_hat,_ = get_cam_frame_coord(x, Tcam_inv, ind)
    theta = x[2]

    # A equals xs_hat
    A = xs_hat
    B = zs_hat
    A_B2 = A/B**2
    B_inv = 1/B
    c = cos(theta)
    s = sin(theta)

    # dux/dvar
    dux_dxr = fx*(-B_inv*s+A_B2*c)
    dux_dyr = fx*(B_inv*c+A_B2*s)
    dux_dtheta = fx*(1+A*A_B2)
    dux_dxs = fx*(B_inv*s-A_B2*c)
    dux_dys = fx*(-B_inv*c-A_B2*s)
    dux_dzs = 0

    # duy/dvar
    tmp = fy*ys_hat/(-zs_hat**2)
    duy_dxr = tmp*(-c)
    duy_dyr = tmp*(-s)
    duy_dtheta = tmp*(-xs_hat)
    duy_dxs = tmp*c
    duy_dys = tmp*s
    duy_dzs = -fy/zs_hat

    Jr = np.array([[dux_dxr, dux_dyr, dux_dtheta],
                   [duy_dxr, duy_dyr, duy_dtheta]])
    
    Jl = np.array([[dux_dxs, dux_dys, dux_dzs],
                   [duy_dxs, duy_dys, duy_dzs]])
    
    N = 3+3*4
    H = np.zeros((2,N))
    H[:,:3] = Jr
    i = landmark_ind
    H[:,3+i*3:3+i*3+3] = Jl

    return H

## Confirm Jacobian calculations

In [2]:
from sympy import symbols, Matrix, BlockMatrix
import numpy as np
from sympy import cos as c
from sympy import sin as s
from sympy import diff

In [3]:
xr, yr, t, xs, ys, zs = symbols("x_r y_r \\theta X_s Y_s Z_s")
hc = symbols("h_c")
fx, fy, px, py = symbols("f_x f_y p_x p_y")

In [5]:
Rrobot = Matrix([[c(t), -s(t), 0],
                 [s(t),  c(t), 0],
                 [0,     0,    1]])
trobot = Matrix([[xr],[yr],[0]])
tmp = Matrix(BlockMatrix([Rrobot,trobot]))
tmp 
Trobot = Matrix([[tmp],[0,0,0,1]])
Trobot


Matrix([
[cos(\theta), -sin(\theta), 0, x_r],
[sin(\theta),  cos(\theta), 0, y_r],
[          0,            0, 1,   0],
[          0,            0, 0,   1]])

In [6]:
tmp = Matrix(BlockMatrix([Rrobot.transpose(),-Rrobot.transpose()*trobot]))
Trobot_inv = Matrix([[tmp],[0,0,0,1]])
Trobot_inv


Matrix([
[ cos(\theta), sin(\theta), 0, -x_r*cos(\theta) - y_r*sin(\theta)],
[-sin(\theta), cos(\theta), 0,  x_r*sin(\theta) - y_r*cos(\theta)],
[           0,           0, 1,                                  0],
[           0,           0, 0,                                  1]])

In [7]:
Rbc = Matrix([[0,  0, 1],
                 [-1, 0, 0],
                 [0, -1, 0]])
tbc = Matrix([[0],[0],[hc]])
tmp = Matrix(BlockMatrix([Rbc,tbc]))
tmp 
Tbc = Matrix([[tmp],[0,0,0,1]])
Tbc

Matrix([
[ 0,  0, 1,   0],
[-1,  0, 0,   0],
[ 0, -1, 0, h_c],
[ 0,  0, 0,   1]])

In [8]:
Tbc_inv = Tbc.inv()
Tbc_inv

Matrix([
[0, -1,  0,   0],
[0,  0, -1, h_c],
[1,  0,  0,   0],
[0,  0,  0,   1]])

In [9]:
Tcam_inv = Tbc_inv*Trobot_inv
Tcam_inv

Matrix([
[sin(\theta), -cos(\theta),  0, -x_r*sin(\theta) + y_r*cos(\theta)],
[          0,            0, -1,                                h_c],
[cos(\theta),  sin(\theta),  0, -x_r*cos(\theta) - y_r*sin(\theta)],
[          0,            0,  0,                                  1]])

In [10]:
coord_cam_frame = Tcam_inv*Matrix([[xs],[ys],[zs],[1]])
xs_hat = coord_cam_frame[0]
xs_hat

X_s*sin(\theta) - Y_s*cos(\theta) - x_r*sin(\theta) + y_r*cos(\theta)

In [11]:
ys_hat = coord_cam_frame[1]
ys_hat

-Z_s + h_c

In [12]:
zs_hat = coord_cam_frame[2]
zs_hat

X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta)

## $u_x$

In [13]:
ux = fx*xs_hat/zs_hat + px
ux

f_x*(X_s*sin(\theta) - Y_s*cos(\theta) - x_r*sin(\theta) + y_r*cos(\theta))/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta)) + p_x

In [14]:
diff(ux, xr)

f_x*(X_s*sin(\theta) - Y_s*cos(\theta) - x_r*sin(\theta) + y_r*cos(\theta))*cos(\theta)/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))**2 - f_x*sin(\theta)/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))

In [15]:
diff(ux, yr)

f_x*(X_s*sin(\theta) - Y_s*cos(\theta) - x_r*sin(\theta) + y_r*cos(\theta))*sin(\theta)/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))**2 + f_x*cos(\theta)/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))

In [16]:
diff(ux, t)

f_x*(X_s*sin(\theta) - Y_s*cos(\theta) - x_r*sin(\theta) + y_r*cos(\theta))**2/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))**2 + f_x

In [17]:
diff(ux, xs)

-f_x*(X_s*sin(\theta) - Y_s*cos(\theta) - x_r*sin(\theta) + y_r*cos(\theta))*cos(\theta)/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))**2 + f_x*sin(\theta)/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))

In [18]:
diff(ux, ys)

-f_x*(X_s*sin(\theta) - Y_s*cos(\theta) - x_r*sin(\theta) + y_r*cos(\theta))*sin(\theta)/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))**2 - f_x*cos(\theta)/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))

In [19]:
diff(ux, zs)

0

## $u_y$

In [21]:
uy = fy*ys_hat/zs_hat + py
uy


f_y*(-Z_s + h_c)/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta)) + p_y

In [59]:
diff(uy, xr)

f_y*(-Z_s + h_c)*cos(\theta)/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))**2

In [60]:
diff(uy, yr)

f_y*(-Z_s + h_c)*sin(\theta)/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))**2

In [61]:
diff(uy,t)

f_y*(-Z_s + h_c)*(X_s*sin(\theta) - Y_s*cos(\theta) - x_r*sin(\theta) + y_r*cos(\theta))/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))**2

In [62]:
diff(uy, xs)

-f_y*(-Z_s + h_c)*cos(\theta)/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))**2

In [63]:
diff(uy, ys)

-f_y*(-Z_s + h_c)*sin(\theta)/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))**2

In [22]:
diff(uy, zs)

-f_y/(X_s*cos(\theta) + Y_s*sin(\theta) - x_r*cos(\theta) - y_r*sin(\theta))